In [ ]:
import mlflow
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

mlflow.set_tracking_uri('http://34.90.207.177:5000')

RUN_ID = 'acf70a0372e449d0843db50821c67bfa'

In [ ]:
client = mlflow.MlflowClient()

In [ ]:
runs = client.search_runs(['1'])

In [ ]:
rids = [r.info.run_id for r in runs]

In [ ]:
metrics = {}

for rid in rids:
    eval = [(m.step, m.value) for m in client.get_metric_history(rid, 'eval_loss')]
    train = [(m.step, m.value) for m in client.get_metric_history(rid, 'loss')]
    dftr = pd.DataFrame(train, columns=['step', 'loss']).assign(split="train")
    dfte = pd.DataFrame(eval, columns=['step', 'loss']).assign(split="eval")
    all = pd.concat([dftr, dfte])
    all["rid"] = rid
    metrics[rid] = all

In [ ]:
metrics
#metrics = pd.concat(metrics.values())

In [ ]:
df_metric = pd.concat(metrics.values())

df_metric.loc[lambda df: df["split"] == "eval"].sort_values('loss')

In [ ]:
fig, ax = plt.subplots(figsize=(12, 5))

for rid, df in metrics.items():
    if rid == 'acf70a0372e449d0843db50821c67bfa':
        c = 'red'
        alpha = 1
    else:
        c = 'grey'
        alpha = 0.5
    sns.lineplot(x="step", y="loss", data=df.loc[lambda df: df["split"] == "eval"], ax=ax, color=c, alpha=alpha)

In [ ]:
fig, ax = plt.subplots(figsize=(12, 5))
ax = sns.lineplot(x="step", y="loss", hue="split", data=all, ax=ax)
ax.set_title("Train / eval loss")

In [ ]:
run = mlflow.get_run()

In [ ]:
run.data.metrics